# Project Setup
* This section describes how to create and configure a project
* This is the same as creating a new project in the editor and going through all of the steps.

* When a user creates a project with client.create_project() the project is not ready for labeling.
    * An ontology must be set
    * datasets must be attached

In [1]:
!pip install labelbox

In [2]:
from labelbox import Client, Project, LabelingFrontend
from labelbox.schema.ontology import Tool, OntologyBuilder
from getpass import getpass
import os

In [3]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [4]:
# Set this to a project that is already set up
PROJECT_ID = "ckm4xyfncfgja0760vpfdxoro"
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

In [5]:
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

### Identify project, dataset, and ontology
* Pick the project to setup
* Dataset(s) to attach to that project
* Configure the ontology for the project

In [6]:
# Use bounding boxes to label cats
ontology_builder = OntologyBuilder(
    tools=[Tool(name="cat", tool=Tool.Type.BBOX)])
project = client.create_project(name="my_new_project")
dataset = client.create_dataset(name="my_new_dataset")
# Add data_rows since this is a new dataset (see basics/data_rows.ipynb for more information on this)
test_img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/Kitano_Street_Kobe01s5s4110.jpg/2560px-Kitano_Street_Kobe01s5s4110.jpg"
dataset.create_data_row(row_data=test_img_url)

In [7]:
# Unless you are using a custom editor you should always use the following editor:
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))

In [8]:
# Note that you can use any dataset or ontology even if they already exist.
existing_project = client.get_project(PROJECT_ID)
# We are not using this, but it is possible to copy the ontology to the new project
ontology = existing_project.ontology()

### Setup and attach dataset
* Setting up a project will add an ontology and will enable labeling to begin
* Attaching dataset(s) will add all data_rows belonging to the dataset to the queue.

In [9]:
project.setup(editor, ontology_builder.asdict())
# Could also do if ontology is a normalized ontology
# project.setup(editor, ontology.normalized)

In [10]:
# Run this for each dataset we want to attach
project.datasets.connect(dataset)
# project.datasets.connect(another_dataset)

### Review

In [11]:
# Note setup_complete will be None if it fails.
print(project.setup_complete)
print(project.ontology)
print([ds.name for ds in project.datasets()])

2021-03-17 12:35:00+00:00
['my_new_dataset']


In [12]:
print(f"https://app.labelbox.com/projects/{project.uid}")